In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *


In [2]:
# Direct Address
@flag(name='hideDisconnected')
@hideAtom(selector='NoneType + DirectAddress') # Hide empty slots
@inferredEdge(selector="{k : int, v : (object - NoneType) | (some t : tuple | t.t1 not in NoneType and t.t1 = v and t.t0 = k)}", name='maps-to')
@group(selector='tuple.t0', name='slot')
@group(selector='tuple.t1', name='value')
@attribute(field='t0')
@attribute(field='t1')
class DirectAddress:
    def __init__(self, m): self.T = [None]*m
    def insert(self, k, v): self.T[k] = (k, v)
    def search(self, k):    return self.T[k]
    def delete(self, k):    self.T[k] = None


In [3]:
da = DirectAddress(30)

for k in ["abc", "def", "ghi"]:
    da.insert(hash(k) % 30, k)

diagram(da)

# Now Chained Hash Table

In [4]:
# Minimal CLRS-style chained hash table (element-pointer API)
# Buckets are doubly-linked lists with a per-bucket SENTINEL for O(1) delete-by-pointer.

@flag(name='hideDisconnected')
# Hide prev, since we have next
@hideField(selector='Node', field='prev')
@attribute(field='key')
@attribute(field='val')
@cyclic(selector='{x,y : Node | x->y in next}', direction='clockwise')
class Node:
    __slots__ = ("key", "val", "prev", "next")
    def __init__(self, key, val=None):
        self.key = key
        self.val = val
        self.prev = None
        self.next = None
    def __repr__(self):
        return f"Node(key={self.key}, val={self.val})"

@hideAtom(selector='NoneType + {x : Node | x = x.next and x = x.prev}') # Hide empty slots
@attribute(field='m')
class HashTableChaining:
    def __init__(self, m=8):
        self.m = m
        self.T = [self._new_sentinel() for _ in range(m)]

    # ---- internals ----
    def _new_sentinel(self):
        s = Node(key=None, val=None)
        s.prev = s.next = s          # circular sentinel
        return s

    def _h(self, k):
        return hash(k) % self.m      # abstract h(.) in CLRS; fine for a minimal demo

    # ---- CLRS ops ----
    # CHAINED-HASH-SEARCH(T, k) -> pointer to element or None
    def search(self, k):
        s = self.T[self._h(k)]
        x = s.next
        while x is not s:
            if x.key == k:
                return x
            x = x.next
        return None

    # CHAINED-HASH-INSERT(T, x)  (insert element object)
    # Head insert (like CLRS); does NOT check duplicates.
    def insert(self, x: Node):
        s = self.T[self._h(x.key)]
        # splice x right after sentinel (head insert)
        x.next = s.next
        x.prev = s
        s.next.prev = x
        s.next = x

    # CHAINED-HASH-DELETE(T, x)  (delete by pointer) -> bool
    # O(1) because we have prev/next.
    def delete(self, x: Node):
        if x.prev is None or x.next is None:
            return False  # not currently in a table/chain
        x.prev.next = x.next
        x.next.prev = x.prev
        x.prev = x.next = None
        return True

    # (Optional) convenience method: delete by key using search
    def delete_key(self, k):
        x = self.search(k)
        return self.delete(x) if x else False

    # (Optional) quick view
    def __str__(self):
        lines = []
        for i, s in enumerate(self.T):
            items, x = [], s.next
            while x is not s:
                items.append(f"{x.key}:{x.val}")
                x = x.next
            lines.append(f"[{i}] " + " -> ".join(items) if items else f"[{i}] ·")
        return "\n".join(lines)

# ---- Example usage ----
T = HashTableChaining(m=5)
a = Node(12, "A"); b = Node(7, "B"); c = Node(12, "C")  # duplicate key
d = Node(42, "D"); e = Node(100, "E")  # duplicate key
T.insert(a); T.insert(b); T.insert(c); T.insert(d); T.insert(e)

diagram(T)
